In [1]:
import torch 
from torch import nn
import torch.nn.functional as F


import numpy as np

In [2]:
with open('happy_birthday.txt', 'r', encoding='utf8') as f:
    text = f.read()
    
all_characters = set(text)
all_characters = sorted(all_characters)
print(all_characters)
# nums --> letters
decoder = dict(enumerate(all_characters))
# letters --> nums
encoder = {char: index for index, char in decoder.items()}

# build list of all encoded chars for text
encoded_text = np.array([encoder[char] for char in text])

['\n', ' ', '!', '*', ',', '-', '.', '0', '1', '2', '4', '5', '6', '8', ':', '?', '«', '»', 'А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ж', 'З', 'И', 'К', 'Л', 'М', 'Н', 'О', 'П', 'Р', 'С', 'Т', 'У', 'Х', 'Ц', 'Ч', 'Ш', 'Э', 'Ю', 'Я', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'х', 'ц', 'ч', 'ш', 'щ', 'ы', 'ь', 'э', 'ю', 'я', '–', '—']


In [3]:
def one_hot_encoder(encoded_text, num_unique_chars):
    
    # encoded_text --> batch of encoded text
    # num_unique_chars --> len(set(text))
    
    one_hot = np.zeros((encoded_text.size, num_unique_chars))
    one_hot = one_hot.astype(np.float32)
    one_hot[np.arange(one_hot.shape[0]), encoded_text.flatten()] = 1.0
    one_hot = one_hot.reshape((*encoded_text.shape, num_unique_chars))
    
    return one_hot

In [4]:
class CharModel(nn.Module):
    
    def __init__(self, all_chars, num_hidden=256, num_layers=4,drop_prob=0.5,use_gpu=False):
        
        
        # SET UP ATTRIBUTES
        super().__init__()
        self.drop_prob = drop_prob
        self.num_layers = num_layers
        self.num_hidden = num_hidden
        self.use_gpu = use_gpu
        
        #CHARACTER SET, ENCODER, and DECODER
        self.all_chars = all_chars
        self.decoder = dict(enumerate(all_chars))
        self.encoder = {char: ind for ind,char in decoder.items()}
        
        
        self.lstm = nn.LSTM(len(self.all_chars), num_hidden, num_layers, dropout=drop_prob, batch_first=True)
        
        self.dropout = nn.Dropout(drop_prob)
        
        self.fc_linear = nn.Linear(num_hidden, len(self.all_chars))
      
    
    def forward(self, x, hidden):
                  
        lstm_output, hidden = self.lstm(x, hidden)
        
        drop_output = self.dropout(lstm_output)
        drop_output = drop_output.contiguous().view(-1, self.num_hidden)
        
        final_out = self.fc_linear(drop_output)

        return final_out, hidden
    
    
    def hidden_state(self, batch_size):
        '''
        Used as separate method to account for both GPU and CPU users.
        '''
        
        if self.use_gpu:
            
            hidden = (torch.zeros(self.num_layers,batch_size,self.num_hidden).cuda(),
                     torch.zeros(self.num_layers,batch_size,self.num_hidden).cuda())
        else:
            hidden = (torch.zeros(self.num_layers,batch_size,self.num_hidden),
                     torch.zeros(self.num_layers,batch_size,self.num_hidden))
        
        return hidden
        

In [5]:
model = CharModel(
    all_chars=all_characters,
    num_hidden=512,
    num_layers=3,
    drop_prob=0.5,
    use_gpu=True,
)

model.load_state_dict(torch.load('hidden512_layers3_happy_birthday.net'))
model.eval()

CharModel(
  (lstm): LSTM(76, 512, num_layers=3, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc_linear): Linear(in_features=512, out_features=76, bias=True)
)

In [6]:
def predict_next_char(model, char, hidden=None, k=1):
    
    encoded_text = model.encoder[char]
    encoded_text = np.array([[encoded_text]])
    encoded_text = one_hot_encoder(encoded_text, len(model.all_chars))
    inputs = torch.from_numpy(encoded_text)
    
    if model.use_gpu:
        inputs = inputs.cuda()
        
    hidden = tuple([state.data for state in hidden])
    
    lstm_out, hidden = model(inputs, hidden)
    
    probs = F.softmax(lstm_out, dim=1).data
    
    if model.use_gpu:
        probs = probs.cpu()
        
    probs, index_positions = probs.topk(k)
    
    index_positions = index_positions.numpy().squeeze()
    
    probs = probs.numpy().flatten()
    
    probs = probs / probs.sum()
    
    char = np.random.choice(index_positions, p=probs)
    
    return model.decoder[char], hidden
    
    

In [7]:
def generate_text(model, size, seed='Дело было ', k=1):
    
    if model.use_gpu:
        model.cuda()
    else:
        model.cpu()
        
    model.eval()
    
    output_chars = [c for c in seed]
    
    hidden = model.hidden_state(1)
    
    for char in seed:
        char, hidden = predict_next_char(model, char, hidden, k=k)
        
    output_chars.append(char)
    
    for i in range(size):
        
        char, hidden = predict_next_char(model, output_chars[-1], hidden, k=k)
        
        output_chars.append(char)
        
    return ''.join(output_chars)

In [8]:
print(generate_text(model, 3000, seed='Стих о ', k=2))

Стих о эЮз5ыХЦэЮлУХмБ*—э–эЖ–зсэглУЗэызСЮлУОЦэ,э6зыз:ХЦэЮХЖХдБчзэУлКУзСЧзэьыИ2ЖУзэ–2льЮХмБ0зЖзКИэВЖзэРлЖЦЖэУлШХКлЖУзБчХ:ОэРИВ:Олэ–эСОШУОэьзЮХмБ0И2ЖЗэ–эСОШУОэ2зЯИЖ2Ж–ИлЖэЖл5лэИЮХВХмБкэл2РОэ–зШКзСУздэЧХЧэКзСУзэВХ!лнБ0И2ЖЗэ5лЮ—дэЯлВХРОэЖл5Цэ2зьыллЖмБуэ2эУОКэУлэ2ЖыХ:УзэУОВльзмБ1лРХлКэЯыз2ЖздэзЖэЮИ:ОБоЮзыз–ЗЦдэ2ВХ2ЖЗЦдэЮз5ызЖ—мБчлэЯзКУОэьзыл2ЖлсэОэЯылЧыХ2УлснБ0зэ–ХСУз2ЖОэ2–злсБюУэЮлРОЖ2ЦэЖл5лэЮыИШлсмБжИЮЗэШЮзыз–ХдэУлэ2ЧзВХЖЗБкэКУзСл2Ж–зэРлЖБглУЗэызСЮлУЗЦнБ ХКЯХУ2ЧОКдэВЖзэЧОЯОЖБиэылЮОэУлэШУХЖЗдБшР—5Чзсэ2–лЖР—»э2ОЦЖЗнББЛЛЛББ0И2ЖЗэ5ИЮлЖэЯзРзУэ2ВХ2ЖРО–зсдБкэЧыХ2О–—сэ5ИЮ—дэЯылЧРзСУз6эЮлРОдБНЖз5эьзРз–ХэзЖэьзыЦэУлэ2лЮлРХБкэУОЧзьЮХэ2лыЗлШУзэУлэ5зРлРХнББЛЛЛББюУэУлэИызУОЖэВл2ЖЗэКИУЮОыХдэЖХРХУЖРО–дэ–л2лРэОэИКлУмБуэ–эызРОэЮХК2ЧзьзэЧИКОыХэзУдэЧХСлЖ2ЦдэУлэЯыл–ШХсЮлУмББЛЛЛББюУэУлэИызУОЖэВл2ЖЗэКИУЮОыХдэЖХРХУЖРО–дэ–эСОШУЗэ–2льзпэРлЖЦЖэьзЮХмБчзэ–2лэСэзЖУл2Рз2ЗэЧэУОКэ2эИВХ2ЖЗлКБ0ылЧыХ2УзсэР»5–Оэ5зСл2Ж–зпБг–ХэьзРИ5ЦэУлСУзэ–зыЧИ»ЖэрБг–И6эУлСУ—6э2лыЮлтэЖзыСл2Ж–змББЛЛЛББ8эШХЧзУУ—Кэ5ыХЧзКэЯзШЮыХ–РЦлКБ8эгУлКэызСЮлУО